In [4]:
%load_ext lab_black
# %load_ext memory_profiler

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [5]:
import sys

sys.path.append("../")

In [6]:
from functional import seq
from itertools import product
from magpie.src.analyzer import HTMLElementAnalyzer
from magpie.src.extractor import ContentExtractor
from magpie.src.parser import EntityParser

In [7]:
document = open(
    # "../data/hellokitty-ransomware-lacks-stealth-but-still-strikes-home.html",
    "../data/hafnium-targeting-exchange-servers.html",
    "r",
).read()

In [8]:
analyzer = HTMLElementAnalyzer(document)
extractor = ContentExtractor()
parser = EntityParser(analyzer.keywords)

# pipe

In [9]:
# original
# %memit
seq(analyzer.primary_subtree_leafs).map(
    lambda element: seq(extractor.get_contents(element)).map(
        lambda content: (element, content)
    )
).flatten().starmap(
    lambda element, content: seq(parser.iterscan(content)).map(
        lambda entity: (element, content, entity)
    )
).flatten()

<Element button at 0x13340bcc8>,"Content(property='text', text='\n\t\tShare\t', span=(165, 174))","Text(entity=None, string='\n\t\tShare\t', span=(0, 9))"
<Element button at 0x13340bcc8>,"Content(property='tail', text='\n\t', span=(183, 185))","Text(entity=None, string='\n\t', span=(0, 2))"
<Element a at 0x13340bd88>,"Content(property='text', text='\n\t\t\t\tTwitter\t\t\t', span=(331, 346))","Text(entity=None, string='\n\t\t\t\tTwitter\t\t\t', span=(0, 15))"
<Element a at 0x13340bd88>,"Content(property='tail', text='\n\t\t', span=(350, 353))","Text(entity=None, string='\n\t\t', span=(0, 3))"
<Element a at 0x13340be08>,"Content(property='text', text='\n\t\t\t\tLinkedIn\t\t\t', span=(462, 478))","Text(entity=None, string='\n\t\t\t\tLinkedIn\t\t\t', span=(0, 16))"
<Element a at 0x13340be08>,"Content(property='tail', text='\n\t\t', span=(482, 485))","Text(entity=None, string='\n\t\t', span=(0, 3))"
<Element a at 0x13340be88>,"Content(property='text', text='\n\t\t\t\tFacebook\t\t\t', span=(264, 280))","Text(entity=None, string='\n\t\t\t\tFacebook\t\t\t', span=(0, 16))"
<Element a at 0x13340be88>,"Content(property='tail', text='\n\t\t', span=(284, 287))","Text(entity=None, string='\n\t\t', span=(0, 3))"
<Element a at 0x13340bf08>,"Content(property='text', text='\n\t\t\t\tEmail\t\t\t', span=(306, 319))","Text(entity=None, string='\n\t\t\t\tEmail\t\t\t', span=(0, 13))"
<Element a at 0x13340bf08>,"Content(property='tail', text='\n\t\t', span=(323, 326))","Text(entity=None, string='\n\t\t', span=(0, 3))"


In [10]:
# product
results = (
    seq(analyzer.primary_subtree_leafs)
    .map(lambda element: product([element], extractor.get_contents(element)))
    .flatten()
    .starmap(
        lambda element, content: product([element], [content], parser.iterscan(content))
    )
    .flatten()
    .cache()
)

In [11]:
results.starmap(
    lambda element, content, text: (
        text.string,
        text.entity,
        # element.sourceline,
        element,
        content.property,
        content.span,
        text.span,
    )
)

Share,,<Element button at 0x13340bcc8>,text,"(165, 174)","(0, 9)"
,,<Element button at 0x13340bcc8>,tail,"(183, 185)","(0, 2)"
Twitter,,<Element a at 0x13340bd88>,text,"(331, 346)","(0, 15)"
,,<Element a at 0x13340bd88>,tail,"(350, 353)","(0, 3)"
LinkedIn,,<Element a at 0x13340be08>,text,"(462, 478)","(0, 16)"
,,<Element a at 0x13340be08>,tail,"(482, 485)","(0, 3)"
Facebook,,<Element a at 0x13340be88>,text,"(264, 280)","(0, 16)"
,,<Element a at 0x13340be88>,tail,"(284, 287)","(0, 3)"
Email,,<Element a at 0x13340bf08>,text,"(306, 319)","(0, 13)"
,,<Element a at 0x13340bf08>,tail,"(323, 326)","(0, 3)"


In [12]:
product_by_contents = lambda element: product(
    [element], extractor.get_contents(element)
)

product_by_entities = lambda element, content: product(
    [element], [content], parser.iterscan(content)
)

In [13]:
seq(analyzer.primary_subtree_leafs).map(product_by_contents).flatten().starmap(
    product_by_entities
).flatten().starmap(
    lambda element, content, text: (
        text.string,
        text.entity,
        # element.sourceline,
        element,
        content.property,
        content.span,
        text.span,
    )
)

Share,,<Element button at 0x13340bcc8>,text,"(165, 174)","(0, 9)"
,,<Element button at 0x13340bcc8>,tail,"(183, 185)","(0, 2)"
Twitter,,<Element a at 0x13340bd88>,text,"(331, 346)","(0, 15)"
,,<Element a at 0x13340bd88>,tail,"(350, 353)","(0, 3)"
LinkedIn,,<Element a at 0x13340be08>,text,"(462, 478)","(0, 16)"
,,<Element a at 0x13340be08>,tail,"(482, 485)","(0, 3)"
Facebook,,<Element a at 0x13340be88>,text,"(264, 280)","(0, 16)"
,,<Element a at 0x13340be88>,tail,"(284, 287)","(0, 3)"
Email,,<Element a at 0x13340bf08>,text,"(306, 319)","(0, 13)"
,,<Element a at 0x13340bf08>,tail,"(323, 326)","(0, 3)"


---

# query

In [14]:
from lxml import etree

etree.tounicode(analyzer.primary_subtree_leafs[1])[49:108][0:10]

'AFNIUM%20t'

---

# Labeler

- labeling_pipeline.ipynb
- conllu_datafeeds

In [15]:
# feed = seq.json("../data/hellokitty.feeds.json").to_dict()
feed = seq.json("../data/hafnium.feeds.json").to_dict()

# stix21 = seq.json("../data/hellokitty.stix21.6047944bd5bd6b6f323e59fa.json").to_dict()

---

# check same fields from feed and stix

* ('name',) 	('objects', 'name')
* ('description',) 	('objects', 'description')
* ('indicators', 'description')	('objects', 'name')
* ('indicators', 'description')	('objects', 'description')
* ('indicators', 'title') 	('objects', 'name')
* ('attack_ids',) 	('objects', 'external_references', 'external_id')
* ('references',) 	('objects', 'external_references', 'url') 



---
('attack_ids',) 	('objects', 'external_references', 'external_id')
('tags',) 	('objects', 'name') 

('indicators', 'indicator')    "pattern": "[file:hashes.MD5 = '136bd70f7aa98f52861879d7dca03cf2']",

('indicators', 'description')	('objects', 'description')
('indicators', 'is_active') 	('objects', 'is_family')


---

# AlientVault Feed

In [16]:
from operator import itemgetter

itemgetter(
    "name",  # pulse.name
    "description",  # pulse.description
    "adversary",  # pulse.adversary
    "tags",  # pulse.tags
    "targeted_countries",  # pulse.targeted_countries
    "malware_families",  # pulse.malware_families
    "attack_ids",  # pulse.attack_ids
    "industries",  # pulse.industries
)(feed)

('HAFNIUM targeting Exchange Servers with 0-day exploits',
 '"Microsoft has detected multiple 0-day exploits being used to attack on-premises versions of Microsoft Exchange Server in limited and targeted attacks. In the attacks observed, the threat actor used these vulnerabilities to access on-premises Exchange servers which enabled access to email accounts, and allowed installation of additional malware to facilitate long-term access to victim environments. Microsoft Threat Intelligence Center (MSTIC) attributes this campaign with high confidence to HAFNIUM, a group assessed to be state-sponsored and operating out of China, based on observed victimology, tactics and procedures."',
 'HAFNIUM',
 ['webshell',
  'aspx webshell',
  'azure sentinel',
  'hafnium',
  'microsoft',
  'exchange',
  'server',
  'powercat',
  'covenant',
  'procdump',
  'lsass',
  'service',
  'powershell',
  'cn_apt',
  'China',
  'email',
  'Exploit',
  'UNC2643',
  'UNC2640',
  'UNC2639',
  'CVE-2021-26858',
  

## Pulse

In [17]:
from collections import namedtuple


class PulseLabel(object):

    Label = namedtuple("Label", ["type", "group", "role"])

    # pre-defined labels
    Name = Label(("pulse", "name"), 0, None)
    Description = Label(("pulse", "description"), 0, None)
    Adversary = Label(("pulse", "adversary"), 0, None)
    AttackId = Label(("pulse", "attack_id"), 0, None)
    Tag = Label(("pulse", "tag"), 0, None)
    TargetedCountry = Label(("pulse", "targeted_country"), 0, None)
    MalwareFamily = Label(("pulse", "malware_family"), 0, None)
    Industry = Label(("pulse", "industry"), 0, None)

    def __init__(self, pulse_field: dict):
        self.field = pulse_field

    def select_by(self, key_name):
        value = self.field.get(key_name)
        return None if value is "" else value  # check and replace "" with None

    @property
    def name(self):
        label = self.select_by("name"), self.Name
        return label

    @property
    def description(self):
        label = self.select_by("description"), self.Description
        return label

    @property
    def adversary(self):
        label = self.select_by("adversary"), self.Adversary
        return label

    @property
    def tags(self):
        labels = product(self.select_by("tags"), [self.Tag])
        return labels

    @property
    def targeted_countries(self):
        labels = product(self.select_by("targeted_countries"), [self.TargetedCountry])
        return labels

    @property
    def malware_families(self):
        labels = product(self.select_by("malware_families"), [self.MalwareFamily])
        return labels

    @property
    def attack_ids(self):
        labels = product(self.select_by("attack_ids"), [self.AttackId])
        return labels

    @property
    def industries(self):
        labels = product(self.select_by("industries"), [self.Industry])
        return labels

    def get_labels(self):
        labels = [self.name, self.description, self.adversary]
        labels = (
            labels
            + list(self.tags)
            + list(self.targeted_countries)
            + list(self.malware_families)
            + list(self.attack_ids)
            + list(self.industries)
        )
        return filter(lambda label: label[0] != None, labels)

In [18]:
p = PulseLabel(feed)

pulse_dict = seq(p.get_labels()).to_dict()
pulse_dict

{'HAFNIUM targeting Exchange Servers with 0-day exploits': Label(type=('pulse', 'name'), group=0, role=None),
 '"Microsoft has detected multiple 0-day exploits being used to attack on-premises versions of Microsoft Exchange Server in limited and targeted attacks. In the attacks observed, the threat actor used these vulnerabilities to access on-premises Exchange servers which enabled access to email accounts, and allowed installation of additional malware to facilitate long-term access to victim environments. Microsoft Threat Intelligence Center (MSTIC) attributes this campaign with high confidence to HAFNIUM, a group assessed to be state-sponsored and operating out of China, based on observed victimology, tactics and procedures."': Label(type=('pulse', 'description'), group=0, role=None),
 'HAFNIUM': Label(type=('pulse', 'adversary'), group=0, role=None),
 'webshell': Label(type=('pulse', 'tag'), group=0, role=None),
 'aspx webshell': Label(type=('pulse', 'tag'), group=0, role=None),
 

---

# Indicator

In [19]:
class IndicatorLabel(object):

    Label = namedtuple("Label", ["type", "group", "role"])

    def __init__(self, enumerated_group: int, indicator_field: dict):
        self.group = enumerated_group
        self.field = indicator_field

    def select_by(self, key_name):
        value = self.field.get(key_name)
        return None if value is "" else value  # check and replace "" with None

    @property
    def indicator(self):

        label_type = self.select_by("type")
        content = self.select_by("content")

        # check indicator's field isn't rule
        if label_type is "YARA" and content.startswith("rule"):
            yara = self.select_by("content")
            label = self.Label(("indicator", label_type), self.group, "rule")
            return yara, label
        else:
            indicator = self.select_by("indicator")
            label = self.Label(
                ("indicator", label_type), self.group, self.select_by("role")
            )
            return indicator, label

    @property
    def title(self):
        title = self.select_by("title")
        label = self.Label(("indicator", "title"), self.group, None)
        return title, label

    @property
    def description(self):
        description = self.select_by("description")
        label = self.Label(("indicator", "description"), self.group, None)
        return description, label

    def get_labels(self):
        labels = [self.indicator, self.title, self.description]
        return filter(lambda label: label[0] != None, labels)

---

# Labeler

In [20]:
from enum import Enum
from collections import ChainMap
from functools import lru_cache
from textdistance import jaro

In [21]:
class FeedLabeler(object):
    class MatchThreshold(Enum):
        exactly_match = 1
        fuzzy_match = 0.9

    def __init__(self, feed):
        self.feed = feed

    @property
    def pulse(self):
        pulse = PulseLabel(self.feed)
        pulse = seq(pulse.get_labels()).cache()
        return pulse.to_dict()

    @property
    def indicators(self):
        indicators = self.feed.get("indicators")
        indicators = (
            seq(indicators)
            .enumerate(1)
            .map(lambda indicator: IndicatorLabel(*indicator))
            .map(lambda indicator: indicator.get_labels())
            .flatten()
            .cache()
        )
        return indicators.to_dict()

    @property
    def labels(self):
        return ChainMap(self.pulse, self.indicators)

    def match_label(self, string, method="fuzzy_match"):

        # match
        matching_results = (
            seq(self.labels.items())
            .starmap(lambda k, v: (jaro.normalized_similarity(k, string), v))
            .filter(
                lambda similar_string: similar_string[0]
                >= self.MatchThreshold[method].value
            )
            .sorted(lambda similar_string: similar_string[0], reverse=True)
            .cache()
        )

        if matching_results.empty():
            label = None
        else:
            label = matching_results.starmap(lambda similarity, label: label).first()

        return label

In [22]:
fl = FeedLabeler(feed)

fl.labels

ChainMap({'HAFNIUM targeting Exchange Servers with 0-day exploits': Label(type=('pulse', 'name'), group=0, role=None), '"Microsoft has detected multiple 0-day exploits being used to attack on-premises versions of Microsoft Exchange Server in limited and targeted attacks. In the attacks observed, the threat actor used these vulnerabilities to access on-premises Exchange servers which enabled access to email accounts, and allowed installation of additional malware to facilitate long-term access to victim environments. Microsoft Threat Intelligence Center (MSTIC) attributes this campaign with high confidence to HAFNIUM, a group assessed to be state-sponsored and operating out of China, based on observed victimology, tactics and procedures."': Label(type=('pulse', 'description'), group=0, role=None), 'HAFNIUM': Label(type=('pulse', 'adversary'), group=0, role=None), 'webshell': Label(type=('pulse', 'tag'), group=0, role=None), 'aspx webshell': Label(type=('pulse', 'tag'), group=0, role=Non

---

---

# Test

In [23]:
fl.labels.get("T1005")

In [24]:
fl.labels.get("T100")

In [25]:
fl.match_label("T1005")

In [26]:
fl.match_label("HAFNIUM", method="exactly_match")

Label(type=('pulse', 'adversary'), group=0, role=None)

In [27]:
fl.match_label("HAFNIUM", method="fuzzy_match")

Label(type=('pulse', 'adversary'), group=0, role=None)

In [28]:
fl.match_label(
    "HelloKitty is a ransomware family that emerged in late 2020. While it lacks the sophistication of some of the more well-known families such as Ryuk, REvil, and Conti, it has nevertheless struck some notable targets, including CEMIG0. In this post, Sentinel Labs analyzes a recent HelloKitty sample and outlines the basic behaviors and traits associated with this family of ransomware."
)

---

# Test

In [29]:
testing_yara = {
    "id": 2883057223,
    "indicator": "0f6e708599b716dd3b24fdf6c75837d49b61fdd1",
    "type": "YARA",
    "created": "2021-03-03T15:23:29",
    "content": 'rule HKTL_PS1_PowerCat_Mar21 {   \n      meta:   \n         description = "Detects PowerCat hacktool"   \n         author = "Florian Roth"   \n         reference = "https://github.com/besimorhino/powercat"   \n         date = "2021-03-02"   \n         hash1 = "c55672b5d2963969abe045fe75db52069d0300691d4f1f5923afeadf5353b9d2"   \n      strings:   \n         $x1 = "powercat -l -p 8000 -r dns:10.1.1.1:53:c2.example.com" ascii fullword   \n         $x2 = "try{[byte[]]$ReturnedData = $Encoding.GetBytes((IEX $CommandToExecute 2>&1 | Out-String))}" ascii fullword   \n      \n         $s1 = "Returning Encoded Payload..." ascii   \n         $s2 = "$CommandToExecute =" ascii fullword   \n         $s3 = "[alias(\\"Execute\\")][string]$e=\\"\\"," ascii   \n      condition:   \n         uint16(0) == 0x7566 and   \n         filesize < 200KB and   \n         1 of ($x*) or 3 of them   \n   }',
    "title": "HKTL_PS1_PowerCat_Mar21",
    "description": "Detects PowerCat hacktool",
    "expiration": "",
    "is_active": 1,
    "role": "",
}

---

# Fuzzy Join

In [30]:
seq(analyzer.primary_subtree_leafs).map(product_by_contents).flatten().starmap(
    product_by_entities
).flatten().starmap(
    lambda element, content, text: (
        text.string,
        text.entity,
        # element.sourceline,
        element,
        content.property,
        content.span,
        text.span,
    )
).starmap(
    lambda string, entity, element, prop, content_span, text_span: (
        string,
        entity,
        element,
        prop,
        content_span,
        text_span,
        fl.match_label(string, method="fuzzy_match"),
    )
).distinct().filter(
    lambda x: x[6] != None
)

"Microsoft has detected multiple 0-day exploits being used to attack on-premises versions of Microsoft Exchange Server in limited and targeted attacks. In the attacks observed, the threat actor used these vulnerabilities to access on-premises Exchange servers which enabled access to email accounts, and allowed installation of additional malware to facilitate long-term access to victim environments. Microsoft Threat Intelligence Center (MSTIC) attributes this campaign with high confidence to",,<Element p at 0x1333f9648>,text,"(3, 498)","(0, 495)","Label(type=('pulse', 'description'), group=0, role=None)"
HAFNIUM,,<Element a at 0x1333f9688>,text,"(103, 110)","(0, 7)","Label(type=('pulse', 'adversary'), group=0, role=None)"
CVE-2021-26855,CVE,<Element p at 0x1333f96c8>,text,"(3, 160)","(50, 64)","Label(type=('pulse', 'tag'), group=0, role=None)"
CVE-2021-26857,CVE,<Element p at 0x1333f96c8>,text,"(3, 160)","(66, 80)","Label(type=('pulse', 'tag'), group=0, role=None)"
CVE-2021-26858,CVE,<Element p at 0x1333f96c8>,text,"(3, 160)","(82, 96)","Label(type=('pulse', 'tag'), group=0, role=None)"
CVE-2021-27065,CVE,<Element p at 0x1333f96c8>,text,"(3, 160)","(102, 116)","Label(type=('pulse', 'tag'), group=0, role=None)"
Azure Sentinel,,<Element a at 0x1333f9808>,text,"(69, 83)","(0, 14)","Label(type=('pulse', 'tag'), group=0, role=None)"
Covenant,,<Element a at 0x1333f99c8>,text,"(34, 42)","(0, 8)","Label(type=('pulse', 'tag'), group=0, role=None)"
CVE-2021-26855,CVE,<Element a at 0x1333f9b88>,text,"(63, 77)","(0, 14)","Label(type=('pulse', 'tag'), group=0, role=None)"
CVE-2021-26857,CVE,<Element a at 0x1333f9c08>,text,"(63, 77)","(0, 14)","Label(type=('pulse', 'tag'), group=0, role=None)"


---

---

# Pipe

# get parent-child elements labels

Example:

```html
    <p>
        "Game studio CD Projekt Red recently "
        <a href="https://<url>" target="..." rel="...">disclosed</a>
        " that it became a victim of a targeted, highly-impactful ransomware. In the days following the disclosure, it was revealed that the ransomware family most likely behind the attack was “HelloKitty”.?
    </p>
 ```

```python
class FeedLabeler(object):
    
    class MatchThreshold(Enum):
        exactly_match = 1
        fuzzy_match = 0.9
    
    def __init__(self, feed):
        self.feed = feed

    @property
    def pulse(self):
        pulse = PulseLabel(self.feed)
        pulse = seq(pulse.get_labels()).cache()
        return pulse.to_dict()

    @property
    def indicators(self):
        indicators = self.feed.get("indicators")
        indicators = (
            seq(indicators)
            .enumerate(1)
            .map(lambda indicator: IndicatorLabel(*indicator))
            .map(lambda indicator: indicator.get_labels())
            .flatten()
            .cache()
        )
        return indicators.to_dict()

    @property
    def labels(self):
        return ChainMap(self.pulse, self.indicators)

    def match_label(self, string, method="fuzzy_match"):

        # match
        matching_results = (
            seq(self.labels.items())
            .starmap(lambda k, v: (jaro.normalized_similarity(k, string), v))
            .filter(lambda similar_string: similar_string[0] >= self.MatchThreshold[method].value)
            .sorted(lambda similar_string: similar_string[0], reverse=True)
            .cache()
        )

        if matching_results.empty():
            label = None
        else:
            label = matching_results.starmap(lambda similarity, label: label).first()

        return label
```

In [31]:
class ElementLabeler(object):

    """
    For Text Formatting
    """

    """
    <p>
        "Game studio CD Projekt Red recently "
        <a href="https://<url>" target="..." rel="...">disclosed</a>
        " that it became a victim of a targeted, highly-impactful ransomware.
    </p>
    """

    def __init__(self, printable_elements, feed_labeler):
        self.printable_elements = printable_elements
        self.feed_labeler = feed_labeler

    @property
    def formattingtext_families(self):
        # get ancestor elements which has descendants one in printable elements
        ancestors = seq(self.printable_elements).filter(
            lambda element: seq(element.iterdescendants()).intersection(
                self.printable_elements
            )
        )
        # conbine ancestor element and its descendants
        families = ancestors.map(
            lambda element: [element] + list(element.iterdescendants())
        )
        return families

    @staticmethod
    def extract_text(elements):
        """
        product_by_contents = lambda element: product(
            [element], extractor.get_contents(element)
        )

        product_by_entities = lambda element, content: product(
            [element], [content], parser.iterscan(content)
        )
        """

        text = (
            seq(elements)
            .map(product_by_contents)
            .flatten()
            .starmap(product_by_entities)
            .flatten()
            .starmap(lambda element, content, text: text.string)
            .make_string(" ")
        )

        return text

    def make_labels(self):

        labels = (
            self.formattingtext_families.map(
                lambda elements: (elements, self.extract_text(elements))
            )
            .starmap(lambda elements, string: product(elements, [string]))
            .flatten()
            .starmap(
                lambda element, string: (
                    element,
                    self.feed_labeler.match_label(string, method="fuzzy_match"),
                )
            )
            .filter(lambda element_label: element_label[1] is not None)
            .to_dict()
        )

        return labels

    @property
    @lru_cache()
    def labels(self):
        return self.make_labels()

    def match_label(self, element):
        return self.labels.get(element)

In [32]:
el = ElementLabeler(analyzer.primary_subtree_leafs, fl)

In [33]:
el.labels

{<Element p at 0x1333f9648>: Label(type=('pulse', 'description'), group=0, role=None),
 <Element a at 0x1333f9688>: Label(type=('pulse', 'description'), group=0, role=None)}

In [34]:
seq(analyzer.primary_subtree_leafs).map(product_by_contents).flatten().starmap(
    product_by_entities
).flatten().starmap(
    lambda element, content, text: (
        text.string,
        text.entity,
        # element.sourceline,
        element,
        content.property,
        content.span,
        text.span,
    )
).starmap(
    lambda string, entity, element, prop, content_span, text_span: (
        string,
        entity,
        element,
        prop,
        content_span,
        text_span,
        fl.match_label(string, method="fuzzy_match"),
        el.match_label(element),
    )
).distinct().filter(
    lambda x: x[6] != None or x[7] != None
)

"Microsoft has detected multiple 0-day exploits being used to attack on-premises versions of Microsoft Exchange Server in limited and targeted attacks. In the attacks observed, the threat actor used these vulnerabilities to access on-premises Exchange servers which enabled access to email accounts, and allowed installation of additional malware to facilitate long-term access to victim environments. Microsoft Threat Intelligence Center (MSTIC) attributes this campaign with high confidence to",,<Element p at 0x1333f9648>,text,"(3, 498)","(0, 495)","Label(type=('pulse', 'description'), group=0, role=None)","Label(type=('pulse', 'description'), group=0, role=None)"
,,<Element p at 0x1333f9648>,tail,"(739, 740)","(0, 1)",,"Label(type=('pulse', 'description'), group=0, role=None)"
HAFNIUM,,<Element a at 0x1333f9688>,text,"(103, 110)","(0, 7)","Label(type=('pulse', 'adversary'), group=0, role=None)","Label(type=('pulse', 'description'), group=0, role=None)"
", a group assessed to be state-sponsored and operating out of China, based on observed victimology, tactics and procedures.",,<Element a at 0x1333f9688>,tail,"(114, 237)","(0, 123)",,"Label(type=('pulse', 'description'), group=0, role=None)"
CVE-2021-26855,CVE,<Element p at 0x1333f96c8>,text,"(3, 160)","(50, 64)","Label(type=('pulse', 'tag'), group=0, role=None)",
CVE-2021-26857,CVE,<Element p at 0x1333f96c8>,text,"(3, 160)","(66, 80)","Label(type=('pulse', 'tag'), group=0, role=None)",
CVE-2021-26858,CVE,<Element p at 0x1333f96c8>,text,"(3, 160)","(82, 96)","Label(type=('pulse', 'tag'), group=0, role=None)",
CVE-2021-27065,CVE,<Element p at 0x1333f96c8>,text,"(3, 160)","(102, 116)","Label(type=('pulse', 'tag'), group=0, role=None)",
Azure Sentinel,,<Element a at 0x1333f9808>,text,"(69, 83)","(0, 14)","Label(type=('pulse', 'tag'), group=0, role=None)",
Covenant,,<Element a at 0x1333f99c8>,text,"(34, 42)","(0, 8)","Label(type=('pulse', 'tag'), group=0, role=None)",


In [36]:
seq(analyzer.primary_subtree_leafs).map(product_by_contents).flatten().starmap(
    product_by_entities
).flatten().starmap(
    lambda element, content, text: (
        text.string,
        text.entity,
        # element.sourceline,
        element,
        content.property,
        content.span,
        text.span,
    )
).starmap(
    lambda string, entity, element, prop, content_span, text_span: (
        string,
        entity,
        element,
        prop,
        content_span,
        text_span,
        el.match_label(element)
        if el.match_label(element) is not None
        else fl.match_label(string, method="fuzzy_match"),
    )
).distinct().filter(
    lambda x: x[6] != None
)

"Microsoft has detected multiple 0-day exploits being used to attack on-premises versions of Microsoft Exchange Server in limited and targeted attacks. In the attacks observed, the threat actor used these vulnerabilities to access on-premises Exchange servers which enabled access to email accounts, and allowed installation of additional malware to facilitate long-term access to victim environments. Microsoft Threat Intelligence Center (MSTIC) attributes this campaign with high confidence to",,<Element p at 0x1333f9648>,text,"(3, 498)","(0, 495)","Label(type=('pulse', 'description'), group=0, role=None)"
,,<Element p at 0x1333f9648>,tail,"(739, 740)","(0, 1)","Label(type=('pulse', 'description'), group=0, role=None)"
HAFNIUM,,<Element a at 0x1333f9688>,text,"(103, 110)","(0, 7)","Label(type=('pulse', 'description'), group=0, role=None)"
", a group assessed to be state-sponsored and operating out of China, based on observed victimology, tactics and procedures.",,<Element a at 0x1333f9688>,tail,"(114, 237)","(0, 123)","Label(type=('pulse', 'description'), group=0, role=None)"
CVE-2021-26855,CVE,<Element p at 0x1333f96c8>,text,"(3, 160)","(50, 64)","Label(type=('pulse', 'tag'), group=0, role=None)"
CVE-2021-26857,CVE,<Element p at 0x1333f96c8>,text,"(3, 160)","(66, 80)","Label(type=('pulse', 'tag'), group=0, role=None)"
CVE-2021-26858,CVE,<Element p at 0x1333f96c8>,text,"(3, 160)","(82, 96)","Label(type=('pulse', 'tag'), group=0, role=None)"
CVE-2021-27065,CVE,<Element p at 0x1333f96c8>,text,"(3, 160)","(102, 116)","Label(type=('pulse', 'tag'), group=0, role=None)"
Azure Sentinel,,<Element a at 0x1333f9808>,text,"(69, 83)","(0, 14)","Label(type=('pulse', 'tag'), group=0, role=None)"
Covenant,,<Element a at 0x1333f99c8>,text,"(34, 42)","(0, 8)","Label(type=('pulse', 'tag'), group=0, role=None)"


---

```python
formating_text_label = (
    seq(analyzer.primary_subtree_leafs)
    .filter(
        lambda element: seq(element.iterdescendants()).intersection(
            analyzer.primary_subtree_leafs
        )
    )
    .map(lambda element: [element] + list(element.iterdescendants()))
    .map(
        lambda elements: (
            elements,
            seq(elements)
            .map(product_by_contents)
            .flatten()
            .starmap(product_by_entities)
            .flatten()
            .starmap(lambda element, content, text: text.string)
            .make_string(" "),
        )
    )
    .starmap(lambda elements, string: product(elements, [string]))
    .flatten()
    .starmap(
        lambda element, string: (element, fl.match_label(string, method="fuzzy_match"))
    )
    .filter(lambda element_label: element_label[1] is not None)
    .to_dict()
)

formating_text_label
```

```python
seq(analyzer.primary_subtree_leafs).map(product_by_contents).flatten().starmap(
    product_by_entities
).flatten().starmap(
    lambda element, content, text: (
        text.string,
        text.entity,
        # element.sourceline,
        element,
        content.property,
        content.span,
        text.span,
    )
).starmap(
    lambda string, entity, element, prop, content_span, text_span: (
        string,
        entity,
        element,
        prop,
        content_span,
        text_span,
        fl.match_label(string, method="fuzzy_match"),
        formating_text_label.get(element),
    )
).distinct().filter(
    lambda x: x[6] != None or x[7] != None
)
```

---

In [ ]:
lxml

---